# Evolving a Lunar Lander with differentiable Genetic Programming

## Installation
To install the required libraries run the command:

In [1]:
#!pip install -r requirements.txt


Imports from the standard genepro-multi library are done here. Any adjustments (e.g. different operators) should be made in the notebook. For example:

```
class SmoothOperator(Node):
  def __init__(self):
    super(SmoothOperator,self).__init__()
    self.arity = 1
    self.symb = "SmoothOperator"

  def _get_args_repr(self, args):
    return self._get_typical_repr(args,'before')

  def get_output(self, X):
    c_outs = self._get_child_outputs(X)
    return np.smoothOperation(c_outs[0])

  def get_output_pt(self, X):
    c_outs = self._get_child_outputs_pt(X)
    return torch.smoothOperation(c_outs[0])
```

In [2]:
import gymnasium as gym

from genepro.node_impl import *
from genepro.evo import Evolution
from genepro.node_impl import Constant

import torch
import torch.optim as optim

import random
import os
import copy
from collections import namedtuple, deque

import matplotlib.pyplot as plt
from matplotlib import animation

## Reinforcement Learning Setup
Here we first setup the Gymnasium environment. Please see https://gymnasium.farama.org/environments/box2d/lunar_lander/ for more information on the environment. 

Then a memory buffer is made. This is a buffer in which state transitions are stored. When the buffer reaches its maximum capacity old transitions are replaced by new ones.

A frame buffer is initialised used to later store animation frames of the environment.

In [3]:
env = gym.make("LunarLander-v2", render_mode="rgb_array")

In [4]:
Transition = namedtuple("Transition", ("state", "action", "next_state", "reward"))


class ReplayMemory(object):
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

    def __iadd__(self, other):
        self.memory += other.memory
        return self

    def __add__(self, other):
        self.memory = self.memory + other.memory
        return self

## Fitness Function

Here you get to be creative. The default setup evaluates 5 episodes of 300 frames. Think of what action to pick and what fitness function to use. The Multi-tree takes an input of $n \times d$ where $n$ is a batch of size 1.

In [5]:
def fitness_function_pt(multitree, num_episodes=5, episode_duration=300, ignore_done=False, render=False):
    memory = ReplayMemory(10000)
    episode_rewards = []
    if render:
        frames = []

    # print(multitree.get_readable_repr())
    for _ in range(num_episodes):
        # get initial state of the environment
        observation = env.reset()
        observation = observation[0]
        rewards = []
        for _ in range(episode_duration):
            if render:
                frames.append(env.render())
            input_sample = torch.from_numpy(observation.reshape((1, -1))).float()
            action = torch.argmax(multitree.get_output_pt(input_sample)).detach()
            observation, reward, terminated, truncated, info = env.step(action.item())
            rewards.append(reward)
            output_sample = torch.from_numpy(observation.reshape((1, -1))).float()
            memory.push(input_sample, torch.tensor([[action.item()]]), output_sample, torch.tensor([reward]))
            if (terminated or truncated) and not ignore_done:
                break
        episode_rewards.append(np.sum(rewards))

    # Get the average reward over all episodes
    fitness = episode_rewards
    if render:
        return fitness, memory, frames
    return fitness, memory

def rollout_function(multitree, num_episodes=5, episode_duration=300, ignore_done=False, render=False):
    env = gym.make("LunarLander-v2", render_mode="rgb_array")

    episode_rewards = []
    if render:
        frames = []

    for _ in range(num_episodes):
        # get initial state of the environment
        observation = env.reset()
        observation = observation[0]
        rewards = []
        for _ in range(episode_duration):
            if render:
                frames.append(env.render())
            input_sample = observation.reshape((1, -1))
            action = np.argmax(multitree.get_output(input_sample))
            observation, reward, terminated, truncated, info = env.step(action.item())
            rewards.append(reward)
            if (terminated or truncated) and not ignore_done:
                break
        episode_rewards.append(np.sum(rewards))

    # Get the average reward over all episodes
    episode_rewards = np.array(episode_rewards)
    if render:
        return episode_rewards, [], frames
    return episode_rewards, []

def fitness_baseline(episode_rewards, pop):
    return np.mean(episode_rewards)

def fitness_function_mean_len_sqrt_std(episode_rewards, pop):
    return np.mean(episode_rewards) - len(pop) - np.std(episode_rewards)**0.5

def fitness_function_mean_len_std(episode_rewards, pop):
    return np.mean(episode_rewards) - len(pop) - np.std(episode_rewards)

In [6]:
### USED TO STORE THE EXPERIMENT DICTIONARY
import inspect
import itertools
import pickle


def serialize_functions_in_dict(dictionary):
    for key, value in dictionary.items():
        if inspect.isfunction(value) or inspect.ismethod(value):
            dictionary[key] = value.__name__
        elif isinstance(value, list):
            for i, item in enumerate(value):
                if isinstance(item, dict):
                    value[i] = serialize_functions_in_dict(item)
                elif inspect.isfunction(item) or inspect.ismethod(item):
                    value[i] = item.__name__
                elif isinstance(item, Node):
                    value[i] = item.symb
        elif isinstance(value, dict):
            dictionary[key] = serialize_functions_in_dict(value)
    return dictionary

### USED TO CREATE THE EXPERIMENT DICTIONARY
def grid_search_params(params_dict):
    """
    Given a dictionary of hyperparameters, if a value is a list, loop over all values
    and create a grid search.
    """
    param_keys = params_dict.keys()
    param_values = params_dict.values()
    param_combinations = list(itertools.product(*[v if isinstance(v, list) else [v] for v in param_values]))
    for combination in param_combinations:
        yield dict(zip(param_keys, combination))

In [7]:
# Save the gen as a pickle file in the gens folder
def save_and_evaluate_evo_generations(evo, rollout_function, experiment_name, num_episodes=10):
    generation_evo_fitnesses = []
    generation_test_fitnesses = []
    for i, gen in enumerate(evo.best_of_gens):
        if i == 0:
            continue

        episode_rewards, _ = rollout_function(gen, num_episodes=num_episodes)
        evo_fitness_mean, evo_fitness_std = round(np.mean(gen.fitnesses), 3), round(np.std(gen.fitnesses), 3)
        test_fitness_mean, test_fitness_std  = round(np.mean(episode_rewards), 3), round(np.std(episode_rewards), 3)
        print(f"Best of Generation {i}: evo fitness:{evo_fitness_mean}+/-{evo_fitness_std} \t test_fitness:{test_fitness_mean}+/-{test_fitness_std}")
        
        generation_evo_fitnesses.append(gen.fitnesses)
        generation_test_fitnesses.append(episode_rewards)
        # create the gens folder if it doesn't exist
        os.makedirs(f"./experiments/{experiment_name}/gen/", exist_ok=True) 
        with open(f"./experiments/{experiment_name}/gen/gen_{i}_{evo_fitness_mean}_{test_fitness_mean}.pickle", "wb") as f:
            pickle.dump(gen, f)

    np.save(f"./experiments/{experiment_name}/generation_evo_fitnesses.npy", generation_evo_fitnesses)
    np.save(f"./experiments/{experiment_name}/generation_test_fitnesses.npy", generation_test_fitnesses)   
    return generation_evo_fitnesses, generation_test_fitnesses

def plot_evo_test_fitnesses(evo_fitnesses, test_fitnesses, experiment_name):
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.set_title(f"Fitnesses: {experiment_name}")
    ax.set_xlabel("Generation")
    ax.set_ylabel("Fitness")
    ax.plot(np.arange(len(evo_fitnesses)), [np.mean(gen) for gen in evo_fitnesses], label="evo_fitness", color='tab:blue')
    ax.fill_between(np.arange(len(evo_fitnesses)), [np.mean(gen) - np.std(gen) for gen in evo_fitnesses], [np.mean(gen) + np.std(gen) for gen in evo_fitnesses], alpha=0.2, color='tab:blue')
    ax.plot(np.arange(len(test_fitnesses)), [np.mean(gen) for gen in test_fitnesses], label="test_fitness", color='tab:orange')
    ax.fill_between(np.arange(len(test_fitnesses)), [np.mean(gen) - np.std(gen) for gen in test_fitnesses], [np.mean(gen) + np.std(gen) for gen in test_fitnesses], alpha=0.2, color='tab:orange')
    ax.legend()
    plt.savefig(f"./experiments/{experiment_name}/{experiment_name}.png")
    plt.close()

## Evolution Setup
Here the leaf and internal nodes are defined. Think about the odds of sampling a constant in this default configurations. Also think about any operators that could be useful and add them here. 

Adjust the population size (multiple of 8 if you want to use the standard tournament selection), max generations and max tree size to taste. Be aware that each of these settings can increase the runtime.

#### BASELINE

In [8]:
from copy import deepcopy
import json
from genepro.selection import elitism_selection, tournament_selection
from genepro.variation import coeff_mutation, subtree_crossover, subtree_mutation

experiment_name = "fitness_mean-len-std"
num_features = env.observation_space.shape[0]
evo_settings = {
    "rollout_function": rollout_function,
    "fitness_function": fitness_function_mean_len_std,
    "internal_nodes": [[Plus(), Times(), Div(), Sin(), Sqrt(), Square()]],
    "leaf_nodes": [[Feature(i) for i in range(num_features)] + [Constant()]],
    "n_trees": 4,
    "pop_size": 32,
    "max_gens": 500,
    "init_max_depth": 4,
    "max_tree_size": 128,
    "crossovers": [[{"fun": subtree_crossover, "rate": 0.5}]],
    "mutations": [[{"fun": subtree_mutation, "rate": 0.5}]],
    "coeff_opts": [[{"fun": coeff_mutation, "rate": 0.5}]],
    "selection": {"fun": tournament_selection, "kwargs": {"tournament_size": 4}},
    # "selection": {"fun": elitism_selection},
    "n_jobs": 8,
    "verbose": True
}

def hpo_evolve(evo_settings, experiment_name):
    hpo_settings = list(grid_search_params(evo_settings))
    for settings in hpo_settings:
        serialized_dict = serialize_functions_in_dict(deepcopy(settings))
        print(serialized_dict)
        
    for i, settings in enumerate(hpo_settings):
        specific_experiment_name = experiment_name + f"_pops{settings['pop_size']}_gens{settings['max_gens']}_mts{settings['max_tree_size']}_cor{settings['crossovers'][0]['rate']}_mutr{settings['mutations'][0]['rate']}_coeffr{settings['coeff_opts'][0]['rate']}"
        os.makedirs(f"./experiments/{specific_experiment_name}", exist_ok=True)
        with open(f"./experiments/{specific_experiment_name}/evo_settings.json", "w") as f:
            serialized_dict = serialize_functions_in_dict(deepcopy(settings))
            json.dump(serialized_dict, f)

        evo_baseline = Evolution(**settings)
        evo_baseline.evolve()

        with open(f"./experiments/{specific_experiment_name}/evolution_class.pickle", "wb") as f:
            pickle.dump(evo_baseline, f)
        
        generation_evo_fitnesses, generation_test_fitnesses = save_and_evaluate_evo_generations(evo_baseline, evo_settings['rollout_function'], specific_experiment_name, num_episodes=5)
        plot_evo_test_fitnesses(generation_evo_fitnesses, generation_test_fitnesses, specific_experiment_name)

for i in range(n_experiments:=4):
    hpo_evolve(evo_settings, experiment_name=f'{experiment_name}_exp{i}')

{'rollout_function': 'rollout_function', 'fitness_function': 'fitness_function_mean_len_std', 'internal_nodes': ['+', '*', '/', 'sin', 'sqrt', '**2'], 'leaf_nodes': ['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'const?'], 'n_trees': 4, 'pop_size': 32, 'max_gens': 500, 'init_max_depth': 4, 'max_tree_size': 128, 'crossovers': [{'fun': 'subtree_crossover', 'rate': 0.5}], 'mutations': [{'fun': 'subtree_mutation', 'rate': 0.5}], 'coeff_opts': [{'fun': 'coeff_mutation', 'rate': 0.5}], 'selection': {'fun': 'tournament_selection', 'kwargs': {'tournament_size': 4}}, 'n_jobs': 8, 'verbose': True}
gen: 1 (4.7s),	best of gen fitness: -153.820; reward:-115.108+/-14.712,	best of gen size: 24
gen: 2 (6.0s),	best of gen fitness: -139.066; reward:-113.393+/-11.673,	best of gen size: 14
gen: 3 (7.7s),	best of gen fitness: -154.243; reward:-73.574+/-66.669,	best of gen size: 14
gen: 4 (9.0s),	best of gen fitness: -157.822; reward:-94.837+/-44.986,	best of gen size: 18
gen: 5 (10.5s),	best of g

C:\Users\caspa\miniconda3\envs\genepromulti\lib\site-packages\gymnasium\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Best of Generation 1: evo fitness:-115.108+/-14.712 	 test_fitness:-63.933+/-59.708
Best of Generation 2: evo fitness:-113.393+/-11.673 	 test_fitness:-125.505+/-21.555
Best of Generation 3: evo fitness:-73.574+/-66.669 	 test_fitness:-131.82+/-22.52
Best of Generation 4: evo fitness:-94.837+/-44.986 	 test_fitness:-104.917+/-55.018
Best of Generation 5: evo fitness:-109.357+/-15.415 	 test_fitness:-118.974+/-20.839
Best of Generation 6: evo fitness:-106.158+/-16.555 	 test_fitness:-127.592+/-12.097
Best of Generation 7: evo fitness:-113.982+/-8.072 	 test_fitness:-117.827+/-9.708
Best of Generation 8: evo fitness:-100.947+/-13.599 	 test_fitness:-112.1+/-13.05
Best of Generation 9: evo fitness:-112.993+/-11.412 	 test_fitness:-135.355+/-9.089
Best of Generation 10: evo fitness:-115.335+/-9.197 	 test_fitness:-154.617+/-35.495
Best of Generation 11: evo fitness:-98.844+/-13.437 	 test_fitness:-127.454+/-23.402
Best of Generation 12: evo fitness:-103.38+/-11.514 	 test_fitness:-109.945+

C:\Users\caspa\Documents\01_SoftwareDevelopmentLocation\VS Code\genepromulti\genepro\node_impl.py:97: RuntimeWarning: overflow encountered in square
  return c_outs[0]**2


Best of Generation 19: evo fitness:-86.227+/-4.249 	 test_fitness:-94.297+/-13.763
Best of Generation 20: evo fitness:-85.855+/-7.335 	 test_fitness:-106.454+/-12.147
Best of Generation 21: evo fitness:-68.524+/-6.88 	 test_fitness:-85.555+/-24.956
Best of Generation 22: evo fitness:-66.035+/-15.205 	 test_fitness:-85.154+/-19.351
Best of Generation 23: evo fitness:-66.095+/-9.449 	 test_fitness:-80.783+/-13.504
Best of Generation 24: evo fitness:-53.235+/-14.15 	 test_fitness:-58.343+/-21.421
Best of Generation 25: evo fitness:-46.836+/-27.599 	 test_fitness:-80.295+/-21.001
Best of Generation 26: evo fitness:-40.25+/-17.736 	 test_fitness:-65.221+/-29.748
Best of Generation 27: evo fitness:-75.335+/-6.65 	 test_fitness:-64.447+/-18.352
Best of Generation 28: evo fitness:-46.093+/-21.692 	 test_fitness:-63.316+/-19.037
Best of Generation 29: evo fitness:-31.243+/-11.315 	 test_fitness:-91.755+/-26.492
Best of Generation 30: evo fitness:-49.765+/-11.647 	 test_fitness:-64.944+/-21.358


C:\Users\caspa\miniconda3\envs\genepromulti\lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


gen: 473 (774.6s),	best of gen fitness: -46.920; reward:-21.555+/-13.365,	best of gen size: 12
gen: 474 (776.9s),	best of gen fitness: -57.571; reward:-18.775+/-20.796,	best of gen size: 18
gen: 475 (778.5s),	best of gen fitness: -22.666; reward:-5.525+/-5.141,	best of gen size: 12
gen: 476 (780.3s),	best of gen fitness: -36.863; reward:0.900+/-15.763,	best of gen size: 22
gen: 477 (782.0s),	best of gen fitness: -43.951; reward:-19.016+/-13.935,	best of gen size: 11
gen: 478 (783.9s),	best of gen fitness: -36.365; reward:-9.083+/-22.282,	best of gen size: 5
gen: 479 (786.6s),	best of gen fitness: -35.522; reward:-1.946+/-13.576,	best of gen size: 20
gen: 480 (788.4s),	best of gen fitness: -37.081; reward:-16.571+/-14.510,	best of gen size: 6
gen: 481 (791.4s),	best of gen fitness: -46.261; reward:-27.331+/-12.929,	best of gen size: 6
gen: 482 (793.1s),	best of gen fitness: -38.912; reward:-4.464+/-23.448,	best of gen size: 11
gen: 483 (794.7s),	best of gen fitness: -32.004; reward:-12.

C:\Users\caspa\Documents\01_SoftwareDevelopmentLocation\VS Code\genepromulti\genepro\node_impl.py:73: RuntimeWarning: overflow encountered in divide
  protected_div = sign_b * c_outs[0] / (1e-9 + np.abs(c_outs[1]))


Best of Generation 305: evo fitness:-17.07+/-14.775 	 test_fitness:-26.355+/-18.799
Best of Generation 306: evo fitness:-6.964+/-6.961 	 test_fitness:-20.836+/-12.314
Best of Generation 307: evo fitness:-8.3+/-15.174 	 test_fitness:-20.076+/-22.461
Best of Generation 308: evo fitness:-8.334+/-8.149 	 test_fitness:-40.377+/-10.694
Best of Generation 309: evo fitness:-2.999+/-17.643 	 test_fitness:-15.798+/-15.709
Best of Generation 310: evo fitness:-5.086+/-7.74 	 test_fitness:-14.002+/-15.643
Best of Generation 311: evo fitness:5.462+/-23.052 	 test_fitness:-4.142+/-18.26
Best of Generation 312: evo fitness:-20.211+/-10.372 	 test_fitness:-22.962+/-13.459
Best of Generation 313: evo fitness:0.441+/-22.615 	 test_fitness:-21.1+/-35.287
Best of Generation 314: evo fitness:-10.915+/-13.279 	 test_fitness:-12.633+/-28.164
Best of Generation 315: evo fitness:-6.691+/-8.938 	 test_fitness:-46.951+/-25.515
Best of Generation 316: evo fitness:-8.546+/-16.324 	 test_fitness:-24.892+/-21.136
Bes

# Vizualize the different experiments.

## Make an animation
Here the best evolved individual is selected and one episode is rendered. Make sure to save your lunar landers over time to track progress and make comparisons.

In [9]:
# # gist to save gif from https://gist.github.com/botforge/64cbb71780e6208172bbf03cd9293553
# def save_frames_as_gif(frames, path="./", filename="evolved_lander.gif"):
#     plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi=72)
#     patch = plt.imshow(frames[0])
#     plt.axis("off")

#     def animate(i):
#         patch.set_data(frames[i])

#     anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval=50)
#     anim.save(path + filename, writer="imagemagick", fps=60)


# frames = []
# avg_fitness, frames = get_test_score(evo.best_of_gens[-1], num_episodes=5, episode_duration=300, seed=5, render=True)
# print("Average fitness of the render is: ", avg_fitness)
# env.close()
# save_frames_as_gif(frames)

## Play animation

<img src="evolved_lander.gif" width="750">

## Optimisation
The coefficients in the multi-tree aren't optimised. Here Q-learning (taken from https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) is used to optimise the weights further. Incorporate coefficient optimisation in training your agent(s). Coefficient Optimisation can be expensive. Think about how often you want to optimise, when, which individuals etc.

In [10]:
# batch_size = 128
# GAMMA = 0.99

# constants = best.get_subtrees_consts()

# if len(constants) > 0:
#     optimizer = optim.AdamW(constants, lr=1e-3, amsgrad=True)

# for _ in range(500):
#     if len(constants) > 0 and len(evo.memory) > batch_size:
#         target_tree = copy.deepcopy(best)

#         transitions = evo.memory.sample(batch_size)
#         batch = Transition(*zip(*transitions))

#         non_final_mask = torch.tensor(
#             tuple(map(lambda s: s is not None, batch.next_state)), dtype=torch.bool
#         )

#         non_final_next_states = torch.cat(
#             [s for s in batch.next_state if s is not None]
#         )
#         state_batch = torch.cat(batch.state)
#         action_batch = torch.cat(batch.action)
#         reward_batch = torch.cat(batch.reward)

#         state_action_values = best.get_output_pt(state_batch).gather(1, action_batch)
#         next_state_values = torch.zeros(batch_size, dtype=torch.float)
#         with torch.no_grad():
#             next_state_values[non_final_mask] = (
#                 target_tree.get_output_pt(non_final_next_states).max(1)[0].float()
#             )

#         expected_state_action_values = (next_state_values * GAMMA) + reward_batch

#         criterion = nn.SmoothL1Loss()
#         loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

#         # Optimize the model
#         optimizer.zero_grad()
#         loss.backward()
#         torch.nn.utils.clip_grad_value_(constants, 100)
#         optimizer.step()

# print(best.get_readable_repr())
# print(get_test_score(best))

In [11]:
# frames = []
# fitness_function_pt(
#     best, num_episodes=1, episode_duration=500, render=True, ignore_done=False
# )
# env.close()
# save_frames_as_gif(frames, filename="evolved_lander_RL.gif")

<img src="evolved_lander_RL.gif" width="750">